Define global variables
===

In [1]:
from utils import *
import transformations.transformations as TF
from scipy.spatial.transform import Rotation as R

# Geometry parameters
R_earth = 6378; # radius of earth in kilometers
R_moon = 1737; # radius of moon in kilometers
distance_earth_moon = 384400; # in kilometers
rot_speed_earth = 2*pi/24; # rad/h
rot_speed_moon = 2 * pi / (27.32 * 24); # rad/h
phi_synodic_earth = radians(28.58); # deg 
phi_synodic_moon = radians(6.68); # deg
G = 8.6467e-13; # Gravity constant in km^3*kg^-1*h^-2

# # Transformation of earth, moon w.r.t synodic
T_synodic_earth_0 = R.from_euler('y', phi_synodic_earth, degrees=False).as_matrix()
ang_vel_earth = np.dot(T_synodic_earth_0, np.array([0, 0, 1]))
T_synodic_moon_0 = R.from_euler('y', phi_synodic_moon, degrees=False).as_matrix()
ang_vel_moon = np.dot(T_synodic_moon_0, np.array([0, 0, 1]))

# Physical parameters
mass_earth = 5.97219e24  # kg
mass_moon = 7.34767e22  # kg
mass_sc = 49735  # kg (initial mass of spacecraft)
mass_sc_final = 4000  # kg (final mass of spacecraft after fuel consumption)

# Thrust parameters
Ve = 3.6  # Exhaust velocity in km/s
m_dot = 4000  # Mass flow rate in kg/s
thrust_unit_vec = np.array([0, 0, 0])  # Thrust direction vector

# State variables
phi_earth_space = np.deg2rad(60)
theta_earth_space = np.deg2rad(100)
time_stamp = [0]
X_sc, V_sc, thrust_unit_vec = set_initial_position(phi_earth_space, theta_earth_space, R_earth, T_synodic_earth_0)
mass_sc = [49735] # Mass of space craft 
theta_earth = [0]; # angle of earth
theta_moon = [0]; # angle of moon


# Simulation parameters
time_escape_eath = 0.0 
time_to_zero_gra = 0.0
time_land_moon = 0.0
delta_T = 1; # simulation time step (hour)

# Class Space 

In [4]:
x_sc = 100
space_craft = SpaceCraft()
space_craft.step_update()


NameError: name 'x_sc' is not defined